In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re

In [20]:
def madlan(city): #אם שם העיר מורכב מ2 מיים יש להפריד במקף ("הוד-השרון")
    url = "https://www.madlan.co.il/for-sale/" + city + "-ישראל"
    headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/113.0.0.0 Safari/537.36'}
    response = requests.get(url, headers=headers)
    
     # An empty list to store asset details
    details_lst=[]
    
    try:
        if not response.status_code == 200:
            return None
        results_page = BeautifulSoup(response.content, 'html.parser')
        # Find all assets on the page
        assets = results_page.find_all('div', {'data-auto': 'listed-bulletin', 'data-auto-bulletin-type': 'bulletin'})
        # loop over each asset and extract the details
        for asset in assets:
            asset_details = {}
            asset_details['City'] = city

            asset_type = asset.find("div", {"data-auto": "property-class"})
            asset_details['type'] = str(asset_type.text.strip() if asset_type else "-")

            asset_room_number = asset.find("div", {"data-auto": "property-rooms"})
            asset_room_number_string = asset_room_number.text.strip() if asset_room_number else "-"
            asset_details['room_number'] = str(re.sub(r'[^\d.-]', '', asset_room_number_string) if asset_room_number_string else "-")

            asset_area = asset.find("div", {"data-auto": "property-size"})
            asset_area_string = asset_area.text.strip() if asset_area else "-"
            asset_area_numbers_only = re.findall(r'\d+', asset_area_string)
            asset_details['Area'] = int(asset_area_numbers_only[0]) if asset_area_numbers_only else "-"

            asset_address = asset.find('div', {'data-auto': 'property-address'})
            asset_address = asset_address.get_text(strip=True) if asset_address else "-"
            Street_and_num = asset_address.split(',')[0] if asset_address != "N/A" else "-"
            Street_and_num = Street_and_num.split()
            Street = ''
            num = ''
            for part in Street_and_num:
                if part.isnumeric():
                    num = num + part
                else:
                    Street = Street + ' ' + part
            asset_details['Street'] = str(Street.strip())
            asset_details['number_in_street'] = int(num) if num != '' else "-"
            asset_details['city_area'] = str(asset_address.split(',')[1].strip() if len(asset_address.split(',')) > 1 else "-")

            price_element = asset.find("div", {"data-auto": "property-price"})
            price_string = price_element.text.strip() if price_element else "-"
            num_match = re.search(r'\d+(?:,\d+)*', price_string)
            asset_details['price'] = int(num_match.group().replace(',', '')) if num_match else "-"

            # Append asset details to the details_lst
            details_lst.append(asset_details)

        df = pd.DataFrame(details_lst)

    except:
        pass
    
    df.to_csv('C:/users/amits/assets.csv', index=False,encoding='utf-8-sig')

    return df


In [21]:
assets_details = madlan('הוד-השרון')
assets_details

,City,type,room_number,Area,Street,number_in_street,city_area,price
0,הוד-השרון,מגרש,-,125,הנחל,13,גיל עמל,369000
1,הוד-השרון,דירה,5,135,הפרדס,-,הדר,4790000
2,הוד-השרון,דו משפחתי,6,300,ביאליק,-,רמתיים,7900000
3,הוד-השרון,דירה,4,-,האודם,-,גרינפארק,3490000
4,הוד-השרון,דירה,4,118,הגאולה,18,רמתיים,3300000
5,הוד-השרון,דו משפחתי,7,300,משושים,9,רמת הדר,9380000
6,הוד-השרון,דירה,5,-,האלון,-,הדר,3365000
7,הוד-השרון,דירה,5,130,גרינברג,-,-,5400000
8,הוד-השרון,דירה,5,132,בן גמלא יהושע,-,מרכז העיר,4180000
9,הוד-השרון,דירה,3,100,הדרור,24,הדר,2970000
